在python中常用的数据分析工具中，我们通常使用pandas软件包。

# 2.2.1 读取数据集

首先创建一个人工数据集，并存储在CSV文件../data/house_tiny.csv中。以其他格式存储的数据也可以通过类似的方式进行处理。下面我们将数据集按行写入CSV文件中。

In [1]:
import os
os.makedirs(os.path.join('..','data'),exist_ok=True)
data_file=os.path.join('..','data','house_tiny.csv')
with open(data_file,'w') as f: # w:	打开一个文件只用于写入。如果该文件已存在则将其覆盖。如果该文件不存在，创建新文件。
    f.write('NumRooms,Alley,Price\n') # 列名
    f.write('NA,Pave,127500\n') # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

要从创建的CSV文件中加载原始数据集，我们导入pandas包并调用read_csv函数。该数据集有四行三列。其中每行描述了房间数量、巷子类型和房屋价格。

In [2]:
import pandas as pd

data=pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


# 2.2.2 处理缺失值
"NaN"项代表缺失值。为了处理缺失的数据，典型的方法包括插值法和删除法，其中插值法用一个替代值弥补缺失值，而删除法则直接忽略缺失值。在这里，我们将考虑插值法。

通过位置索引iloc，我们将data分为inputs和outputs，其中前者为data的前两列，而后者为data的最后一列。对于inputs中缺少的数值，我们用同一列的均值替换"NaN"项。

In [3]:
inputs,outputs=data.iloc[:,0:2],data.iloc[:,2]
inputs=inputs.fillna(inputs.mean())
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


对于inputs中的类别值或离散值，我们将"NaN"视为一个类别。由于“巷子类型”列只接受两种类型的类别值"Pave"和"NaN"，pandas可以自动将此列转换为两列“Alley_Pave”和“Alley_nan”。 巷子类型为“Pave”的行会将“Alley_Pave”的值设置为1，“Alley_nan”的值设置为0。 缺少巷子类型的行会将“Alley_Pave”和“Alley_nan”分别设置为0和1。

In [4]:
inputs=pd.get_dummies(inputs,dummy_na=True) # get_dummies方法主要用于对类别型特征做One-Hot编码（独热编码）
# dummy_na=True：增加一列表示空缺值，如果False就忽略空缺值
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


# 2.2.3 转换为张量格式


In [5]:
import torch

X,y=torch.tensor(inputs.values),torch.tensor(outputs.values)
X,y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

# 2.2.5 练习

创建包含更多行和列的原始数据集。
1. 删除缺失值最多的列
2. 将预处理后的数据集转换为张量格式

In [34]:
with open(data_file,'w') as f:
    f.write('NumRooms,Alley,Ceil,traffic,Price\n') # 列名
    f.write('NA,Pave,NA,Good,127500\n') # 每行表示一个数据样本
    f.write('2,NA,a,NA,106000\n')
    f.write('4,NA,b,NA,178100\n')
    f.write('NA,NA,b,NA,110000\n')
    f.write('NA,Pave,c,NA,340000\n')

In [36]:
data=pd.read_csv(data_file)
print(data)

   NumRooms Alley Ceil traffic   Price
0       NaN  Pave  NaN    Good  127500
1       2.0   NaN    a     NaN  106000
2       4.0   NaN    b     NaN  178100
3       NaN   NaN    b     NaN  110000
4       NaN  Pave    c     NaN  340000


In [37]:
inputs,outputs=data.iloc[:,0:4],data.iloc[:,4]
print(inputs)

   NumRooms Alley Ceil traffic
0       NaN  Pave  NaN    Good
1       2.0   NaN    a     NaN
2       4.0   NaN    b     NaN
3       NaN   NaN    b     NaN
4       NaN  Pave    c     NaN


In [38]:
# data 是通过 pandas.read_csv() 读入的数据
inputs2 = inputs.drop(columns=inputs.isna().sum(axis=0).idxmax())  # axis=0表示缺失值最多的“列”
print(inputs2)

   NumRooms Alley Ceil
0       NaN  Pave  NaN
1       2.0   NaN    a
2       4.0   NaN    b
3       NaN   NaN    b
4       NaN  Pave    c


In [40]:
# 将预处理后的数据集转换为张量格式
target=torch.tensor(outputs.values)

inputs=pd.get_dummies(inputs2,dummy_na=True) # 只会拆分非数值类的
inputs=inputs.fillna(inputs.mean())
print(inputs)

inputs=torch.tensor(inputs.values)
inputs,target

   NumRooms  Alley_Pave  Alley_nan  Ceil_a  Ceil_b  Ceil_c  Ceil_nan
0       3.0           1          0       0       0       0         1
1       2.0           0          1       1       0       0         0
2       4.0           0          1       0       1       0         0
3       3.0           0          1       0       1       0         0
4       3.0           1          0       0       0       1         0


(tensor([[3., 1., 0., 0., 0., 0., 1.],
         [2., 0., 1., 1., 0., 0., 0.],
         [4., 0., 1., 0., 1., 0., 0.],
         [3., 0., 1., 0., 1., 0., 0.],
         [3., 1., 0., 0., 0., 1., 0.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 110000, 340000]))